In [31]:
import numpy as np
import pyodbc
import pandas as pd
import warnings

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=./Databases/aenc.accdb;'
)
cnxn = pyodbc.connect(conn_str)
crsr = cnxn.cursor()
DBNorthWind = {'servername': 'LAPTOP-JCB37LKM\SQLEXPRESS',
               'database': 'NorthWind'}

export_conn_NorthWind = pyodbc.connect('DRIVER={SQL Server};SERVER='+ DBNorthWind['servername'] + ';DATABASE=' + DBNorthWind['database'] + ';Trusted_Connection=yes')

DBAdventureWorks = {'servername': 'LAPTOP-JCB37LKM\SQLEXPRESS',
                    'database': 'AdventureWorks2019'}

export_conn_AdventureWorks = pyodbc.connect('DRIVER={SQL Server};SERVER='+ DBAdventureWorks['servername'] + ';DATABASE=' + DBAdventureWorks['database'] + ';Trusted_Connection=yes')


In [32]:
aenc_df_sales_order = pd.read_sql_query("SELECT * FROM Sales_order",cnxn)
aenc_df_sales_order_item = pd.read_sql_query("SELECT * FROM Sales_order_item",cnxn)

nw_df_orders = pd.read_sql_query("SELECT * FROM Orders",export_conn_NorthWind)
nw_df_orders_details = pd.read_sql_query("SELECT * FROM OrderDetails",export_conn_NorthWind)

adv_salesOrderDetail = pd.read_sql_query("SELECT * FROM Sales.SalesOrderDetail",export_conn_AdventureWorks)
adv_SalesOrderHeader = pd.read_sql_query("SELECT * FROM Sales.SalesOrderHeader",export_conn_AdventureWorks)

C:\Users\jboek\AppData\Local\Temp\ipykernel_57860\420703445.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenc_df_sales_order = pd.read_sql_query("SELECT * FROM Sales_order",cnxn)
C:\Users\jboek\AppData\Local\Temp\ipykernel_57860\420703445.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenc_df_sales_order_item = pd.read_sql_query("SELECT * FROM Sales_order_item",cnxn)
C:\Users\jboek\AppData\Local\Temp\ipykernel_57860\420703445.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_df_orders = pd.read_sql_query("S

In [33]:
aenc_df_sales_order_item

aenc_df_sales_order_item_merged = pd.merge(aenc_df_sales_order_item,aenc_df_sales_order,on='id',how='left')
aenc_df_sales_order_item_merged['Source'] = 'AenC'
aenc_df_sales_order_item_merged['id'] = aenc_df_sales_order_item_merged['id'].astype('int64')
aenc_df_sales_order_item_merged['line_id'] = aenc_df_sales_order_item_merged['line_id'].astype('int64')
# aenc_df_sales_order_item_merged['sales_rep'] = aenc_df_sales_order_item_merged['sales_rep'].astype('int64')
aenc_df_sales_order_item_merged['prod_id'] = aenc_df_sales_order_item_merged['prod_id'].astype('int64')
aenc_df_sales_order_item_merged['quantity'] = aenc_df_sales_order_item_merged['quantity'].astype('int64')
aenc_df_sales_order_item_merged['Source'] = aenc_df_sales_order_item_merged['Source'].astype('str')

aenc_df_sales_order_item_merged

,id,line_id,prod_id,quantity,ship_date,cust_id,order_date,region,sales_rep,Source
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC
1,2001,2,301,12,14-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC
2,2001,3,302,12,14-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC
3,2002,1,400,24,18-Sep-1996 12:00:00 AM,102,18-Mar-1996 12:00:00 AM,Eastern,467,AenC
4,2002,2,401,24,18-Sep-1996 12:00:00 AM,102,18-Mar-1996 12:00:00 AM,Eastern,467,AenC
...,...,...,...,...,...,...,...,...,...,...
1098,2652,2,300,10,01-Dec-1998 12:00:00 AM,174,26-Nov-1998 12:00:00 AM,Central,902,AenC
1099,2653,1,301,0,17-Oct-1999 12:00:00 AM,102,17-Jan-1999 12:00:00 AM,Eastern,148,AenC
1100,2654,1,301,15,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC
1101,2654,2,302,3,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC


In [34]:
products_df = pd.read_csv('./Dimensies/Product.csv')
products_dropped = products_df.drop(columns=['Unnamed: 0','Name','Subcategory','Category'])
products_dropped

,ProductKey,ProductID,Source
0,1,1,AdventureWorks
1,2,2,AdventureWorks
2,3,3,AdventureWorks
3,4,4,AdventureWorks
4,5,316,AdventureWorks
...,...,...,...
586,587,73,NorthWind
587,588,74,NorthWind
588,589,75,NorthWind
589,590,76,NorthWind


In [35]:
aenc_df_sales_order_item_joined_productkey = pd.merge(aenc_df_sales_order_item_merged,products_dropped,left_on=['prod_id','Source'],right_on=['ProductID','Source'])
aenc_df_sales_order_item_joined_productkey

,id,line_id,prod_id,quantity,ship_date,cust_id,order_date,region,sales_rep,Source,ProductKey,ProductID
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC,505,300
1,2006,1,300,48,28-Sep-1996 12:00:00 AM,105,28-Mar-1996 12:00:00 AM,Central,299,AenC,505,300
2,2015,1,300,24,06-Oct-1996 12:00:00 AM,114,30-Apr-1996 12:00:00 AM,Central,949,AenC,505,300
3,2019,1,300,12,13-Aug-1996 12:00:00 AM,118,13-Apr-1996 12:00:00 AM,South,667,AenC,505,300
4,2030,1,300,24,06-Oct-1996 12:00:00 AM,129,06-May-1996 12:00:00 AM,Eastern,299,AenC,505,300
...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2625,3,700,24,09-Nov-1998 12:00:00 AM,122,08-Nov-1998 12:00:00 AM,Eastern,690,AenC,514,700
1099,2635,1,700,72,16-Nov-1998 12:00:00 AM,127,15-Nov-1998 12:00:00 AM,South,129,AenC,514,700
1100,2641,2,700,24,18-Nov-1998 12:00:00 AM,130,18-Nov-1998 12:00:00 AM,Central,949,AenC,514,700
1101,2649,1,700,36,26-Nov-1998 12:00:00 AM,132,25-Nov-1998 12:00:00 AM,Central,690,AenC,514,700


In [36]:
df_employee = pd.read_csv('./Dimensies/Employee.csv')
df_employee_dropped = df_employee.drop(columns=['Unnamed: 0','FirstName','LastName','Address','City','State','Country'])
df_employee_dropped

,EmployeeKey,EmployeeID,Source
0,1,285,AdventureWorks
1,2,283,AdventureWorks
2,3,278,AdventureWorks
3,4,279,AdventureWorks
4,5,277,AdventureWorks
...,...,...,...
96,97,5,NorthWind
97,98,6,NorthWind
98,99,7,NorthWind
99,100,8,NorthWind


In [37]:
aenc_df_sales_order_item_joined_employee = pd.merge(aenc_df_sales_order_item_joined_productkey,df_employee_dropped,left_on=['sales_rep','Source'],right_on=['EmployeeID','Source'])
aenc_df_sales_order_item_joined_employee

,id,line_id,prod_id,quantity,ship_date,cust_id,order_date,region,sales_rep,Source,ProductKey,ProductID,EmployeeKey,EmployeeID
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC,505,300,32,299
1,2006,1,300,48,28-Sep-1996 12:00:00 AM,105,28-Mar-1996 12:00:00 AM,Central,299,AenC,505,300,32,299
2,2030,1,300,24,06-Oct-1996 12:00:00 AM,129,06-May-1996 12:00:00 AM,Eastern,299,AenC,505,300,32,299
3,2049,1,300,24,19-Aug-1996 12:00:00 AM,148,18-Jul-1996 12:00:00 AM,Canada,299,AenC,505,300,32,299
4,2156,1,300,12,13-Aug-1997 12:00:00 AM,157,12-Aug-1997 12:00:00 AM,Central,299,AenC,505,300,32,299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2654,2,302,3,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC,507,302,21,148
1099,2654,3,400,8,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC,508,400,21,148
1100,2003,3,400,12,23-Sep-1996 12:00:00 AM,103,21-Mar-1996 12:00:00 AM,Eastern,1039,AenC,508,400,68,1039
1101,2003,1,500,12,23-Sep-1996 12:00:00 AM,103,21-Mar-1996 12:00:00 AM,Eastern,1039,AenC,510,500,68,1039


In [38]:
df_product_aenc = pd.read_sql_query("SELECT id,unit_price FROM product",cnxn)
df_product_aenc['id'] = df_product_aenc['id'].astype('int64')
aenc_df_sales_order_item_unit_price = pd.merge(aenc_df_sales_order_item_joined_employee,df_product_aenc,left_on='prod_id',right_on='id',how='left')
aenc_toBeFormated = aenc_df_sales_order_item_unit_price
aenc_df_sales_order_item_unit_price

C:\Users\jboek\AppData\Local\Temp\ipykernel_57860\820263215.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_product_aenc = pd.read_sql_query("SELECT id,unit_price FROM product",cnxn)


,id_x,line_id,prod_id,quantity,ship_date,cust_id,order_date,region,sales_rep,Source,ProductKey,ProductID,EmployeeKey,EmployeeID,id_y,unit_price
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,101,14-Mar-1996 12:00:00 AM,Eastern,299,AenC,505,300,32,299,300,9
1,2006,1,300,48,28-Sep-1996 12:00:00 AM,105,28-Mar-1996 12:00:00 AM,Central,299,AenC,505,300,32,299,300,9
2,2030,1,300,24,06-Oct-1996 12:00:00 AM,129,06-May-1996 12:00:00 AM,Eastern,299,AenC,505,300,32,299,300,9
3,2049,1,300,24,19-Aug-1996 12:00:00 AM,148,18-Jul-1996 12:00:00 AM,Canada,299,AenC,505,300,32,299,300,9
4,2156,1,300,12,13-Aug-1997 12:00:00 AM,157,12-Aug-1997 12:00:00 AM,Central,299,AenC,505,300,32,299,300,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2654,2,302,3,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC,507,302,21,148,302,14
1099,2654,3,400,8,29-Oct-1999 12:00:00 AM,105,26-Feb-1999 12:00:00 AM,South,148,AenC,508,400,21,148,400,9
1100,2003,3,400,12,23-Sep-1996 12:00:00 AM,103,21-Mar-1996 12:00:00 AM,Eastern,1039,AenC,508,400,68,1039,400,9
1101,2003,1,500,12,23-Sep-1996 12:00:00 AM,103,21-Mar-1996 12:00:00 AM,Eastern,1039,AenC,510,500,68,1039,500,7


In [39]:
final_aenc_orderdetail_item = aenc_toBeFormated.drop(columns=['ship_date','cust_id','region','ProductID','sales_rep','EmployeeID','id_y'])
final_aenc_orderdetail_item = final_aenc_orderdetail_item.reindex(columns=['id_x','line_id','ProductKey','EmployeeKey','quantity','unit_price','order_date','Source'])
final_aenc_orderdetail_item = final_aenc_orderdetail_item.rename(columns={'id_x' : 'OrderID', 'line_id' : 'OrderLineID', 'quantity': 'Quantity', 'unit_price': 'UnitPrice','order_date' : 'OrderDate'})
final_aenc_orderdetail_item

,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source
0,2001,1,505,32,12,9,14-Mar-1996 12:00:00 AM,AenC
1,2006,1,505,32,48,9,28-Mar-1996 12:00:00 AM,AenC
2,2030,1,505,32,24,9,06-May-1996 12:00:00 AM,AenC
3,2049,1,505,32,24,9,18-Jul-1996 12:00:00 AM,AenC
4,2156,1,505,32,12,9,12-Aug-1997 12:00:00 AM,AenC
...,...,...,...,...,...,...,...,...
1098,2654,2,507,21,3,14,26-Feb-1999 12:00:00 AM,AenC
1099,2654,3,508,21,8,9,26-Feb-1999 12:00:00 AM,AenC
1100,2003,3,508,68,12,9,21-Mar-1996 12:00:00 AM,AenC
1101,2003,1,510,68,12,7,21-Mar-1996 12:00:00 AM,AenC


In [40]:
nw_df_orderDetails_merged = pd.merge(nw_df_orders_details,nw_df_orders,on='OrderID',how='left')
nw_df_orderDetails_merged['Source'] = 'NorthWind'
nw_df_orderDetails_merged

,OrderID,ProductID,UnitPrice,Quantity,Discount,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,Source
0,10248,11,14.00,12,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,NorthWind
1,10248,42,9.80,10,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,NorthWind
2,10248,72,34.80,5,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,NorthWind
3,10249,14,18.60,9,0.00,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,NorthWind
4,10249,51,42.40,40,0.00,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,NorthWind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077,64,33.25,2,0.03,RATTC,1,1998-05-06,1998-06-03,NaT,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,NorthWind
2151,11077,66,17.00,1,0.00,RATTC,1,1998-05-06,1998-06-03,NaT,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,NorthWind
2152,11077,73,15.00,2,0.01,RATTC,1,1998-05-06,1998-06-03,NaT,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,NorthWind
2153,11077,75,7.75,4,0.00,RATTC,1,1998-05-06,1998-06-03,NaT,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,NorthWind


In [41]:
df_employee_new = pd.read_csv('./Dimensies/Employee.csv')
df_product_new = pd.read_csv('./Dimensies/Product.csv')
df_employee_new_northwind = df_employee_new.loc[df_employee_new['Source'] == 'NorthWind']
df_employee_new_northwind.loc[:, 'EmployeeID'] = df_employee_new_northwind['EmployeeID'].astype('int64')



In [42]:
nw_df_orderDetails_merged_employee = pd.merge(nw_df_orderDetails_merged,df_employee_new_northwind,on=['EmployeeID','Source'],how='left')
nw_df_orderDetails_merged_product = pd.merge(nw_df_orderDetails_merged_employee,df_product_new,on=['ProductID','Source'],how='left')
nw_df_orderDetails_merged_product

,OrderID,ProductID,UnitPrice,Quantity,Discount,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,...,LastName,Address,City,State,Country,Unnamed: 0_y,ProductKey,Name,Subcategory,Category
0,10248,11,14.00,12,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,...,Buchanan,14 Garrett Hill,London,England,United Kingdom,524,525,Queso Cabrales,Dairy Products,Food
1,10248,42,9.80,10,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,...,Buchanan,14 Garrett Hill,London,England,United Kingdom,555,556,Singaporean Hokkien Fried Mee,Grains/Cereals,Food
2,10248,72,34.80,5,0.00,VINET,5,1996-07-04,1996-08-01,1996-07-16,...,Buchanan,14 Garrett Hill,London,England,United Kingdom,585,586,Mozzarella di Giovanni,Dairy Products,Food
3,10249,14,18.60,9,0.00,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,...,Suyama,Coventry House\nMiner Rd.,London,England,United Kingdom,527,528,Tofu,Produce,Food
4,10249,51,42.40,40,0.00,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,...,Suyama,Coventry House\nMiner Rd.,London,England,United Kingdom,564,565,Manjimup Dried Apples,Produce,Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077,64,33.25,2,0.03,RATTC,1,1998-05-06,1998-06-03,NaT,...,Davolio,507 - 20th Ave. E.Apt. 2A,Seattle,Washington,United States,577,578,Wimmers gute Semmelknödel,Grains/Cereals,Food
2151,11077,66,17.00,1,0.00,RATTC,1,1998-05-06,1998-06-03,NaT,...,Davolio,507 - 20th Ave. E.Apt. 2A,Seattle,Washington,United States,579,580,Louisiana Hot Spiced Okra,Condiments,Food
2152,11077,73,15.00,2,0.01,RATTC,1,1998-05-06,1998-06-03,NaT,...,Davolio,507 - 20th Ave. E.Apt. 2A,Seattle,Washington,United States,586,587,Röd Kaviar,Seafood,Food
2153,11077,75,7.75,4,0.00,RATTC,1,1998-05-06,1998-06-03,NaT,...,Davolio,507 - 20th Ave. E.Apt. 2A,Seattle,Washington,United States,588,589,Rhönbräu Klosterbier,Beverages,Food


In [43]:
final_nw_orderdetail_item = nw_df_orderDetails_merged_product.drop(columns=['Discount','CustomerID','EmployeeID','RequiredDate','ShipAddress','ShipCity','ShipCountry','ShipName','ShippedDate','ShipPostalCode','ShipRegion','ShipVia','Freight','Unnamed: 0_x','Unnamed: 0_y','FirstName','LastName','Address','City','State','Country','Name','Subcategory','Category'])
final_nw_orderdetail_item = final_nw_orderdetail_item.reindex(columns=['OrderID','ProductID','ProductKey','EmployeeKey','Quantity','UnitPrice','OrderDate','Source'])
final_nw_orderdetail_item = final_nw_orderdetail_item.rename(columns={'ProductID' : 'OrderLineID'})

final_nw_orderdetail_item

,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source
0,10248,11,525,97,12,14.00,1996-07-04,NorthWind
1,10248,42,556,97,10,9.80,1996-07-04,NorthWind
2,10248,72,586,97,5,34.80,1996-07-04,NorthWind
3,10249,14,528,98,9,18.60,1996-07-05,NorthWind
4,10249,51,565,98,40,42.40,1996-07-05,NorthWind
...,...,...,...,...,...,...,...,...
2150,11077,64,578,93,2,33.25,1998-05-06,NorthWind
2151,11077,66,580,93,1,17.00,1998-05-06,NorthWind
2152,11077,73,587,93,2,15.00,1998-05-06,NorthWind
2153,11077,75,589,93,4,7.75,1998-05-06,NorthWind


In [44]:
adv_salesOrderDetail_merged = pd.merge(adv_salesOrderDetail,adv_SalesOrderHeader,on='SalesOrderID',how='left')
adv_salesOrderDetail_merged['Source'] = 'AdventureWorks'
adv_salesOrderDetail_merged_product = pd.merge(adv_salesOrderDetail_merged,df_product_new,on=['ProductID','Source'])

adv_salesOrderDetail_merged_product['SalesPersonID'] = adv_salesOrderDetail_merged_product['SalesPersonID'].astype('object')
adv_salesOrderDetail_merged_product.replace(np.nan,'NULL',inplace=True)
adv_salesOrderDetail_merged_product


,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_x,...,TotalDue,Comment,rowguid_y,ModifiedDate_y,Source,Unnamed: 0,ProductKey,Name,Subcategory,Category
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,B207C96D-D9E6-402B-8470-2CC176C42283,...,23153.2339,NULL,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07,AdventureWorks,280,281,"Mountain-100 Black, 42",Mountain Bikes,Bikes
1,43661,25,4E0A-4F89-AE,4,776,1,2024.994,0.0,8099.976,E8E15B60-3A73-4507-9368-A4FB27B6EE0B,...,36865.8012,NULL,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07,AdventureWorks,280,281,"Mountain-100 Black, 42",Mountain Bikes,Bikes
2,43665,70,19F0-4638-8E,1,776,1,2024.994,0.0,2024.994,0D2A679C-6C8E-4CC2-8A55-B8DB83937AC2,...,16158.6961,NULL,5602C304-853C-43D7-9E79-76E320D476CF,2011-06-07,AdventureWorks,280,281,"Mountain-100 Black, 42",Mountain Bikes,Bikes
3,43670,114,F101-4649-85,1,776,1,2024.994,0.0,2024.994,7E07B269-8004-418E-A426-A38B3526E467,...,6893.2549,NULL,573E52A7-573F-40B6-B695-6F3D242E0C6D,2011-06-07,AdventureWorks,280,281,"Mountain-100 Black, 42",Mountain Bikes,Bikes
4,43672,127,F4B5-48D0-BA,2,776,1,2024.994,0.0,4049.988,4FCF9FC7-3B29-41D0-8BAC-E022BD06BA93,...,6895.4100,NULL,194202AA-8E15-4932-A2C4-7A7AC0713639,2011-06-07,AdventureWorks,280,281,"Mountain-100 Black, 42",Mountain Bikes,Bikes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,75059,121158,NULL,1,713,1,49.990,0.0,49.990,130B86CE-7C17-42C5-B2DA-D2DD0687D9E6,...,134.7437,NULL,1569F6C4-0633-42B4-A154-ED3FE809A37C,2014-07-05,AdventureWorks,217,218,"Long-Sleeve Logo Jersey, S",Jerseys,Clothing
121313,75082,121219,NULL,1,713,1,49.990,0.0,49.990,5CC22934-06C4-46AA-AB64-9AB0246A99D9,...,59.6479,NULL,A588873C-3719-465C-B217-A8B415F7DDF0,2014-07-06,AdventureWorks,217,218,"Long-Sleeve Logo Jersey, S",Jerseys,Clothing
121314,75088,121230,NULL,1,713,1,49.990,0.0,49.990,5C085429-02B3-45C9-ADC8-ED18C64C3589,...,125.9258,NULL,E90D88B8-C4D5-4C1C-AADE-66ECDAD220AE,2014-07-07,AdventureWorks,217,218,"Long-Sleeve Logo Jersey, S",Jerseys,Clothing
121315,51823,41781,2FB1-4AE1-BC,1,897,1,200.052,0.0,200.052,34019C07-4672-4619-9E13-9F3A64847794,...,112685.2793,NULL,ECFC6AD2-F55D-43D2-AF03-C08B55535550,2013-07-07,AdventureWorks,401,402,"LL Touring Frame - Blue, 58",Touring Frames,Components


In [45]:
df_employee_new

,Unnamed: 0,EmployeeKey,EmployeeID,FirstName,LastName,Address,City,State,Country,Source
0,0,1,285,Syed,Abbas,7484 Roundtree Drive,Bothell,Washington,United States,AdventureWorks
1,1,2,283,David,Campbell,2284 Azalea Avenue,Bellevue,Washington,United States,AdventureWorks
2,2,3,278,Garrett,Vargas,10203 Acorn Avenue,Calgary,Alberta,Canada,AdventureWorks
3,3,4,279,Tsvi,Reiter,8291 Crossbow Way,Memphis,Tennessee,United States,AdventureWorks
4,4,5,277,Jillian,Carson,80 Sunview Terrace,Duluth,Minnesota,United States,AdventureWorks
...,...,...,...,...,...,...,...,...,...,...
96,96,97,5,Steven,Buchanan,14 Garrett Hill,London,England,United Kingdom,NorthWind
97,97,98,6,Michael,Suyama,Coventry House\nMiner Rd.,London,England,United Kingdom,NorthWind
98,98,99,7,Robert,King,Edgeham Hollow\nWinchester Way,London,England,United Kingdom,NorthWind
99,99,100,8,Laura,Callahan,4726 - 11th Ave. N.E.,Seattle,Washington,United States,NorthWind


In [46]:
adv_salesOrderDetail_merged_employee = pd.merge(adv_salesOrderDetail_merged_product,df_employee_new,left_on=['SalesPersonID','Source'],right_on=['EmployeeID','Source'],how='left')
adv_salesOrderDetail_merged_employee

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_x,...,Category,Unnamed: 0_y,EmployeeKey,EmployeeID,FirstName,LastName,Address,City,State,Country
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,B207C96D-D9E6-402B-8470-2CC176C42283,...,Bikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43661,25,4E0A-4F89-AE,4,776,1,2024.994,0.0,8099.976,E8E15B60-3A73-4507-9368-A4FB27B6EE0B,...,Bikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43665,70,19F0-4638-8E,1,776,1,2024.994,0.0,2024.994,0D2A679C-6C8E-4CC2-8A55-B8DB83937AC2,...,Bikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43670,114,F101-4649-85,1,776,1,2024.994,0.0,2024.994,7E07B269-8004-418E-A426-A38B3526E467,...,Bikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43672,127,F4B5-48D0-BA,2,776,1,2024.994,0.0,4049.988,4FCF9FC7-3B29-41D0-8BAC-E022BD06BA93,...,Bikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,75059,121158,NULL,1,713,1,49.990,0.0,49.990,130B86CE-7C17-42C5-B2DA-D2DD0687D9E6,...,Clothing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121313,75082,121219,NULL,1,713,1,49.990,0.0,49.990,5CC22934-06C4-46AA-AB64-9AB0246A99D9,...,Clothing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121314,75088,121230,NULL,1,713,1,49.990,0.0,49.990,5C085429-02B3-45C9-ADC8-ED18C64C3589,...,Clothing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121315,51823,41781,2FB1-4AE1-BC,1,897,1,200.052,0.0,200.052,34019C07-4672-4619-9E13-9F3A64847794,...,Components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
adv_sales_final = adv_salesOrderDetail_merged_employee.drop(columns=['CarrierTrackingNumber','ProductID','SpecialOfferID','SalesOrderNumber','RevisionNumber','DueDate','ShipDate','ShipMethodID','ShipToAddressID','Status','OnlineOrderFlag','PurchaseOrderNumber','AccountNumber','CustomerID','SalesPersonID','TerritoryID','BillToAddressID','CreditCardID','CreditCardApprovalCode','CurrencyRateID','SubTotal','TaxAmt','Freight','TotalDue','Comment','rowguid_y','rowguid_x','ModifiedDate_y','ModifiedDate_x','LineTotal','Unnamed: 0_x','Unnamed: 0_y','FirstName','LastName','Address','City','State','Country','Name','Subcategory','Category','EmployeeID','UnitPriceDiscount'])
adv_sales_final = adv_sales_final.rename(columns={'SalesOrderID' : 'OrderID','SalesOrderDetailID' : 'OrderLineID','OrderQty': 'Quantity'} )
adv_sales_final = adv_sales_final.reindex(columns=['OrderID','OrderLineID','ProductKey','EmployeeKey','Quantity','UnitPrice','OrderDate','Source'])
adv_sales_final

,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source
0,43659,1,281,NaN,1,2024.994,2011-05-31,AdventureWorks
1,43661,25,281,NaN,4,2024.994,2011-05-31,AdventureWorks
2,43665,70,281,NaN,1,2024.994,2011-05-31,AdventureWorks
3,43670,114,281,NaN,1,2024.994,2011-05-31,AdventureWorks
4,43672,127,281,NaN,2,2024.994,2011-05-31,AdventureWorks
...,...,...,...,...,...,...,...,...
121312,75059,121158,218,NaN,1,49.990,2014-06-28,AdventureWorks
121313,75082,121219,218,NaN,1,49.990,2014-06-29,AdventureWorks
121314,75088,121230,218,NaN,1,49.990,2014-06-30,AdventureWorks
121315,51823,41781,402,NaN,1,200.052,2013-06-30,AdventureWorks


In [48]:
temp_df, final_aenc_orderdetail_item = final_nw_orderdetail_item.align(final_aenc_orderdetail_item, axis=1)
final_nw_orderdetail_item, adv_sales_final = temp_df.align(adv_sales_final, axis=1)

In [49]:
final_nw_orderdetail_item

,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source
0,10248,11,525,97,12,14.00,1996-07-04,NorthWind
1,10248,42,556,97,10,9.80,1996-07-04,NorthWind
2,10248,72,586,97,5,34.80,1996-07-04,NorthWind
3,10249,14,528,98,9,18.60,1996-07-05,NorthWind
4,10249,51,565,98,40,42.40,1996-07-05,NorthWind
...,...,...,...,...,...,...,...,...
2150,11077,64,578,93,2,33.25,1998-05-06,NorthWind
2151,11077,66,580,93,1,17.00,1998-05-06,NorthWind
2152,11077,73,587,93,2,15.00,1998-05-06,NorthWind
2153,11077,75,589,93,4,7.75,1998-05-06,NorthWind


In [50]:
final_df = pd.concat([final_nw_orderdetail_item, adv_sales_final,final_aenc_orderdetail_item])
final_df


,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source
0,10248,11,525,97.0,12,14.0,1996-07-04 00:00:00,NorthWind
1,10248,42,556,97.0,10,9.8,1996-07-04 00:00:00,NorthWind
2,10248,72,586,97.0,5,34.8,1996-07-04 00:00:00,NorthWind
3,10249,14,528,98.0,9,18.6,1996-07-05 00:00:00,NorthWind
4,10249,51,565,98.0,40,42.4,1996-07-05 00:00:00,NorthWind
...,...,...,...,...,...,...,...,...
1098,2654,2,507,21.0,3,14,26-Feb-1999 12:00:00 AM,AenC
1099,2654,3,508,21.0,8,9,26-Feb-1999 12:00:00 AM,AenC
1100,2003,3,508,68.0,12,9,21-Mar-1996 12:00:00 AM,AenC
1101,2003,1,510,68.0,12,7,21-Mar-1996 12:00:00 AM,AenC


In [51]:
final_df['OrderKey'] = range(1, len(final_df) + 1)
final_df

,OrderID,OrderLineID,ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source,OrderKey
0,10248,11,525,97.0,12,14.0,1996-07-04 00:00:00,NorthWind,1
1,10248,42,556,97.0,10,9.8,1996-07-04 00:00:00,NorthWind,2
2,10248,72,586,97.0,5,34.8,1996-07-04 00:00:00,NorthWind,3
3,10249,14,528,98.0,9,18.6,1996-07-05 00:00:00,NorthWind,4
4,10249,51,565,98.0,40,42.4,1996-07-05 00:00:00,NorthWind,5
...,...,...,...,...,...,...,...,...,...
1098,2654,2,507,21.0,3,14,26-Feb-1999 12:00:00 AM,AenC,124571
1099,2654,3,508,21.0,8,9,26-Feb-1999 12:00:00 AM,AenC,124572
1100,2003,3,508,68.0,12,9,21-Mar-1996 12:00:00 AM,AenC,124573
1101,2003,1,510,68.0,12,7,21-Mar-1996 12:00:00 AM,AenC,124574


In [29]:
DBUnitedOutdoorsDWH = {'servername': 'LAPTOP-JCB37LKM\SQLEXPRESS',
               'database': 'UnitedOutdoorsDWH'}

export_conn_UODwh = pyodbc.connect('DRIVER={SQL Server};SERVER='+ DBUnitedOutdoorsDWH['servername'] + ';DATABASE=' + DBUnitedOutdoorsDWH['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn_UODwh.cursor()
export_cursor


In [30]:
import sys

for index, row in final_df.iterrows():
    try:
        employee_key = 'NULL' if pd.isnull(row['EmployeeKey']) else row['EmployeeKey']
        query = f"INSERT INTO Sales (OrderID, OrderLineID, ProductKey,EmployeeKey,Quantity,UnitPrice,OrderDate,Source) VALUES ({row['OrderID']}, {row['OrderLineID']},{row['ProductKey']},{employee_key},{row['Quantity']},{row['UnitPrice']},'{row['OrderDate']}','{row['Source']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        # If there's an error, print the query for debugging
        print(sys.exc_info()[1])
        print("Query:", query)

export_conn_UODwh.commit()
export_cursor.close()

In [85]:
df_product_new

,Unnamed: 0,ProductKey,ProductID,Name,Subcategory,Category,Source
0,0,1,1,Adjustable Race,NaN,NaN,AdventureWorks
1,1,2,2,Bearing Ball,NaN,NaN,AdventureWorks
2,2,3,3,BB Ball Bearing,NaN,NaN,AdventureWorks
3,3,4,4,Headset Ball Bearings,NaN,NaN,AdventureWorks
4,4,5,316,Blade,NaN,NaN,AdventureWorks
...,...,...,...,...,...,...,...
586,586,587,73,Röd Kaviar,Seafood,Food,NorthWind
587,587,588,74,Longlife Tofu,Produce,Food,NorthWind
588,588,589,75,Rhönbräu Klosterbier,Beverages,Food,NorthWind
589,589,590,76,Lakkalikööri,Beverages,Food,NorthWind


In [89]:
import sys

for index, row in df_product_new.iterrows():
    try:
        name = row['Name'].replace("'", "''")
        query = f"INSERT INTO Product (ProductID, Name, Subcategory,Category,Source) VALUES ({row['ProductID']}, '{name}','{row['Subcategory']}', '{row['Category']}', '{row['Source']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        # If there's an error, print the query for debugging
        print("Query:", query)
        print("Error:", sys.exc_info()[1])

export_conn_UODwh.commit()
export_cursor.close()

In [28]:
export_cursor.close()

In [ ]:
import sys

for index, row in df_employee_new.iterrows():
    try:
        name = row['FirstName'].replace("'", "''")
        query = f"INSERT INTO Employee (EmployeeID, FirstName, LastName,Address,City,State,Country,Source) VALUES ({row['EmployeeID']}, '{name}','{row['LastName']}', '{row['Address']}','{row['City']}','{row['State']}','{row['Country']}', '{row['Source']}')"
        export_cursor.execute(query)
    except pyodbc.Error:
        # If there's an error, print the query for debugging
        print("Query:", query)
        print("Error:", sys.exc_info()[1])

export_conn_UODwh.commit()
export_cursor.close()